# Day9_0: 확률 분포 (Probability Distributions) - 정답

## 📚 실습 퀴즈 정답 및 해설

---

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.random.seed(42)
print("라이브러리 임포트 완료")

---

## Q1. 정규분포 확률 계산 ⭐

**문제**: 제품 무게가 평균 250g, 표준편차 10g인 정규분포를 따릅니다. 무게가 260g 이하일 확률을 계산하세요.

In [ ]:
# Q1 정답
mu = 250
sigma = 10
x = 260

# 정규분포 객체 생성
dist = stats.norm(loc=mu, scale=sigma)

# CDF로 확률 계산
prob = dist.cdf(x)

print(f"P(X <= {x}) = {prob:.4f}")
print(f"약 {prob:.2%}의 제품이 {x}g 이하입니다.")

In [ ]:
# 테스트
assert 0.8 < prob < 0.9, "확률이 예상 범위를 벗어남"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `stats.norm(loc=평균, scale=표준편차)`로 정규분포 객체 생성
- `cdf(x)`는 P(X <= x)를 계산

**핵심 개념**:
- CDF(Cumulative Distribution Function)는 특정 값 이하일 누적 확률
- 260g은 평균보다 1 표준편차 위 (z = 1)
- 68-95-99.7 규칙에서 평균+1std 이하는 약 84%

**대안**:
```python
# 표준화 후 계산
z = (x - mu) / sigma  # z = 1
prob = stats.norm.cdf(z)  # 표준정규분포
```

**흔한 실수**:
- `pdf()`와 `cdf()` 혼동 (pdf는 밀도, cdf는 확률)
- loc/scale 파라미터 순서 혼동

**실무 팁**:
- 품질 관리에서 규격 상한 이하 비율 계산에 활용

---

## Q2. 정규분포 구간 확률 ⭐⭐

**문제**: IQ 점수가 평균 100, 표준편차 15인 정규분포를 따릅니다. IQ가 85~115 사이일 확률을 계산하세요.

In [ ]:
# Q2 정답
mu = 100
sigma = 15
lower = 85
upper = 115

# 정규분포 객체 생성
iq_dist = stats.norm(loc=mu, scale=sigma)

# 구간 확률: P(lower <= X <= upper) = P(X <= upper) - P(X < lower)
prob = iq_dist.cdf(upper) - iq_dist.cdf(lower)

print(f"P({lower} <= IQ <= {upper}) = {prob:.4f}")
print(f"약 {prob:.2%}의 사람이 IQ {lower}~{upper} 범위입니다.")

In [ ]:
# 테스트: 68-95-99.7 규칙에 따라 약 68%
assert 0.65 < prob < 0.70, "68% 규칙에 부합하지 않음"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- 구간 확률 = CDF(상한) - CDF(하한)
- P(a <= X <= b) = P(X <= b) - P(X <= a)

**핵심 개념**:
- 85와 115는 평균에서 각각 -1, +1 표준편차
- 68-95-99.7 규칙: 평균 +/- 1std 내에 약 68% 포함

**대안**:
```python
# scipy.stats.norm.interval() 사용
# 단, 이는 신뢰구간 계산용
```

**흔한 실수**:
- CDF 순서를 반대로 계산 (상한 - 하한이 맞음)
- 표준편차와 분산 혼동

**실무 팁**:
- 정상 범위 비율 계산, 이상치 탐지에 활용

---

## Q3. 이항분포 기본 ⭐⭐

**문제**: 동전을 20번 던질 때 앞면이 정확히 10번 나올 확률을 계산하세요.

In [ ]:
# Q3 정답
n = 20
p = 0.5
k = 10

# 이항분포 객체 생성
binom_dist = stats.binom(n, p)

# pmf로 정확히 k번 나올 확률 계산
prob = binom_dist.pmf(k)

print(f"P(X = {k}) = {prob:.4f}")
print(f"동전 20번 던져 앞면 정확히 10번: {prob:.2%}")

In [ ]:
# 테스트
assert 0.15 < prob < 0.20, "확률이 예상 범위를 벗어남"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- 이산형 분포는 `pmf()` 사용 (pdf가 아님!)
- `stats.binom(n, p).pmf(k)`: n번 시행에서 정확히 k번 성공 확률

**핵심 개념**:
- pmf (Probability Mass Function): 이산형 확률 변수의 확률 함수
- 이항분포: B(n, p)에서 성공 횟수 X의 분포

**대안**:
```python
# 조합 공식 직접 계산
from math import comb
prob = comb(n, k) * (p ** k) * ((1-p) ** (n-k))
```

**흔한 실수**:
- `cdf(k)` 사용 (이는 k 이하 확률)
- `pdf()` 사용 (이산형은 pmf 사용)

**실무 팁**:
- 전환율 분석에서 "정확히 n명 전환" 확률 계산

---

## Q4. 이항분포 응용 ⭐⭐⭐

**문제**: 광고 클릭률이 5%입니다. 500명에게 노출했을 때 30명 이상 클릭할 확률을 계산하세요.

In [ ]:
# Q4 정답
impressions = 500
click_rate = 0.05
min_clicks = 30

# 이항분포 객체 생성
click_dist = stats.binom(impressions, click_rate)

# P(X >= 30) = 1 - P(X <= 29) = 1 - CDF(29)
prob = 1 - click_dist.cdf(min_clicks - 1)

print(f"기대 클릭 수: {click_dist.mean():.1f}명")
print(f"P(X >= {min_clicks}) = {prob:.4f}")
print(f"30명 이상 클릭 확률: {prob:.2%}")

In [ ]:
# 테스트: 기대값 25보다 높은 30 이상은 50% 미만
assert prob < 0.5, "확률이 50% 이상일 수 없음"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- "이상" 확률 = 1 - "미만" 확률
- P(X >= k) = 1 - P(X <= k-1) = 1 - CDF(k-1)

**핵심 개념**:
- CDF는 "이하" 확률이므로 "이상"은 보수 확률 사용
- 기대값 = n * p = 500 * 0.05 = 25명

**대안**:
```python
# sf (survival function) = 1 - cdf
prob = click_dist.sf(min_clicks - 1)
```

**흔한 실수**:
- `1 - cdf(30)` 사용 (이는 31 이상 확률)
- "이상"과 "초과" 혼동

**실무 팁**:
- 광고 성과 목표 달성 확률 예측에 활용

---

## Q5. 포아송분포 기본 ⭐⭐⭐

**문제**: 웹사이트에 분당 평균 8명이 방문합니다. 특정 1분 동안 10명 이상 방문할 확률을 계산하세요.

In [ ]:
# Q5 정답
avg_visitors = 8
threshold = 10

# 포아송분포 객체 생성
visitor_dist = stats.poisson(avg_visitors)

# P(X >= 10) = 1 - P(X <= 9)
prob = 1 - visitor_dist.cdf(threshold - 1)

print(f"분당 평균 방문자: {avg_visitors}명")
print(f"P(X >= {threshold}) = {prob:.4f}")
print(f"10명 이상 방문 확률: {prob:.2%}")

In [ ]:
# 테스트: 평균 8보다 높은 10 이상은 50% 미만
assert prob < 0.5, "평균보다 높은 값의 확률이 50% 이상일 수 없음"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- 포아송분포: 단위 시간 내 사건 발생 횟수 모델링
- P(X >= k) = 1 - P(X <= k-1)

**핵심 개념**:
- 포아송분포: 기댓값 = 분산 = lambda
- 희귀 사건(콜 수, 방문자 수)에 적합

**대안**:
```python
# survival function 사용
prob = visitor_dist.sf(threshold - 1)
```

**흔한 실수**:
- 이항분포와 포아송분포 혼동
- threshold 경계값 처리 오류

**실무 팁**:
- 서버 용량 계획, 콜센터 인력 배치에 활용

---

## Q6. PDF 시각화 ⭐⭐⭐

**문제**: 평균 50, 표준편차가 각각 5, 10, 15인 3개의 정규분포 PDF를 Plotly로 한 그래프에 시각화하세요.

In [ ]:
# Q6 정답
mu = 50
sigmas = [5, 10, 15]
x_range = (10, 90)

# x 값 생성
x = np.linspace(x_range[0], x_range[1], 200)

# Plotly 그래프 생성
fig = go.Figure()

colors = ['blue', 'green', 'red']
for sigma, color in zip(sigmas, colors):
    # 각 분포의 PDF 계산
    pdf_values = stats.norm.pdf(x, loc=mu, scale=sigma)
    
    # 그래프 추가
    fig.add_trace(go.Scatter(
        x=x,
        y=pdf_values,
        mode='lines',
        name=f'sigma = {sigma}',
        line=dict(color=color, width=2)
    ))

# 레이아웃 설정
fig.update_layout(
    title=f'정규분포 PDF 비교 (평균 = {mu})',
    xaxis_title='X',
    yaxis_title='확률 밀도',
    height=450,
    width=700,
    legend=dict(x=0.8, y=0.95)
)

fig.show()

In [ ]:
# 테스트: 그래프가 생성되었는지 확인
assert len(fig.data) == 3, "3개의 분포가 그려져야 함"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. x 값 범위 생성 (`np.linspace`)
2. 각 표준편차에 대해 PDF 계산
3. `go.Scatter`로 선 그래프 추가

**핵심 개념**:
- 표준편차가 클수록 분포가 넓고 낮아짐
- 모든 PDF의 적분값은 1 (면적 보존)

**대안**:
```python
# Plotly Express 사용 (DataFrame 필요)
df = pd.DataFrame({'x': x})
for sigma in sigmas:
    df[f'sigma={sigma}'] = stats.norm.pdf(x, loc=mu, scale=sigma)
fig = px.line(df, x='x', y=[f'sigma={s}' for s in sigmas])
```

**흔한 실수**:
- x 범위를 너무 좁게 설정
- 각 분포의 색상/레이블 누락

**실무 팁**:
- 표준편차 변화에 따른 불확실성 시각화에 활용

---

## Q7. Q-Q Plot 분석 ⭐⭐⭐

**문제**: 다음 데이터가 정규분포를 따르는지 Q-Q plot과 Shapiro-Wilk 검정으로 확인하세요.

In [ ]:
# Q7 정답
data = [12.5, 14.2, 13.8, 15.1, 14.5, 13.2, 14.8, 13.5, 14.1, 15.3,
        13.9, 14.3, 14.0, 13.7, 14.6, 14.4, 13.6, 14.9, 13.3, 14.7]

# 1. Shapiro-Wilk 검정
stat, p_value = stats.shapiro(data)

print("정규성 검정 결과")
print("=" * 50)
print(f"Shapiro-Wilk 통계량: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value > 0.05:
    print("\n결론: 정규분포를 따름 (p > 0.05)")
else:
    print("\n결론: 정규분포가 아님 (p < 0.05)")

In [ ]:
# 2. Q-Q Plot
sorted_data = np.sort(data)
n = len(data)
theoretical_quantiles = stats.norm.ppf(np.arange(1, n + 1) / (n + 1))

# 회귀선 계산
slope, intercept, r_value, _, _ = stats.linregress(theoretical_quantiles, sorted_data)
line_y = slope * theoretical_quantiles + intercept

fig = go.Figure()

# 산점도
fig.add_trace(go.Scatter(
    x=theoretical_quantiles,
    y=sorted_data,
    mode='markers',
    name='데이터',
    marker=dict(color='blue', size=8)
))

# 기준선
fig.add_trace(go.Scatter(
    x=theoretical_quantiles,
    y=line_y,
    mode='lines',
    name=f'기준선 (R2={r_value**2:.3f})',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title='Q-Q Plot',
    xaxis_title='이론적 분위수',
    yaxis_title='샘플 분위수',
    height=450,
    width=550
)

fig.show()

print(f"\nR-squared: {r_value**2:.4f}")
print("점들이 직선에 잘 맞으면 정규분포!")

In [ ]:
# 테스트
assert p_value > 0.05, "이 데이터는 정규분포를 따라야 함"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. Shapiro-Wilk 검정으로 통계적 판단
2. Q-Q plot으로 시각적 확인
3. R-squared로 직선 적합도 확인

**핵심 개념**:
- Shapiro-Wilk: H0="정규분포", p>0.05면 정규분포 가정
- Q-Q plot: 점들이 대각선에 가까우면 정규분포

**대안**:
```python
# scipy.stats.probplot 사용
stats.probplot(data, dist="norm", plot=None)
```

**흔한 실수**:
- p-value 해석 방향 혼동 (p<0.05면 정규분포 기각)
- Q-Q plot 해석 오류

**실무 팁**:
- 분석 전 항상 정규성 검정 수행

---

## Q8. 분포 파라미터 추정 ⭐⭐⭐⭐

**문제**: 다음 매출 데이터로부터 정규분포의 파라미터(평균, 표준편차)를 추정하고, 매출이 200 이상일 확률을 계산하세요.

In [ ]:
# Q8 정답
sales = [150, 165, 180, 175, 160, 155, 170, 185, 175, 165,
         170, 180, 190, 160, 175, 168, 172, 178, 162, 173]

# 파라미터 추정
mu_hat = np.mean(sales)
sigma_hat = np.std(sales, ddof=1)  # 표본 표준편차 (ddof=1)

print("파라미터 추정 결과")
print("=" * 50)
print(f"추정 평균: {mu_hat:.2f}만원")
print(f"추정 표준편차: {sigma_hat:.2f}만원")

# 정규분포 객체 생성
sales_dist = stats.norm(loc=mu_hat, scale=sigma_hat)

# P(X >= 200) = 1 - P(X < 200)
prob = 1 - sales_dist.cdf(200)

print(f"\n매출 200만원 이상 확률: {prob:.4f}")
print(f"약 {prob:.2%}의 확률로 200만원 이상")

In [ ]:
# 테스트
assert 165 < mu_hat < 175, "평균이 예상 범위를 벗어남"
assert prob < 0.1, "200 이상 확률이 10% 미만이어야 함"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 표본 평균, 표본 표준편차 계산 (파라미터 추정)
2. 추정된 파라미터로 정규분포 생성
3. CDF로 확률 계산

**핵심 개념**:
- 표본에서 모집단 파라미터 추정 (점추정)
- ddof=1: 표본 표준편차 (자유도 보정)

**대안**:
```python
# scipy.stats.norm.fit() 사용
mu_hat, sigma_hat = stats.norm.fit(sales)
```

**흔한 실수**:
- ddof=0 사용 (모집단 표준편차)
- 표본 크기가 작을 때 정규분포 가정의 위험성 무시

**실무 팁**:
- 과거 데이터로 미래 성과 확률 예측

---

## Q9. 품질 관리 종합 ⭐⭐⭐⭐

**문제**: 제품 무게가 평균 100g, 표준편차 5g입니다.
1. 규격(90~110g) 내 비율 계산
2. 불량률 계산
3. 10,000개 생산 시 예상 불량 수
4. 불량률을 1% 미만으로 낮추려면 표준편차를 얼마로 줄여야 하는지 계산

In [ ]:
# Q9 정답
mu = 100
sigma = 5
lower_spec = 90
upper_spec = 110
production = 10000
target_defect_rate = 0.01

# 현재 분포
current_dist = stats.norm(loc=mu, scale=sigma)

print("품질 관리 분석")
print("=" * 60)

# 1. 규격 내 비율
in_spec = current_dist.cdf(upper_spec) - current_dist.cdf(lower_spec)
print(f"1. 규격 내 비율: {in_spec:.4f} ({in_spec:.2%})")

# 2. 불량률
defect_rate = 1 - in_spec
print(f"2. 불량률: {defect_rate:.4f} ({defect_rate:.2%})")

# 3. 예상 불량 수
expected_defects = defect_rate * production
print(f"3. 10,000개 생산 시 예상 불량: {expected_defects:.0f}개")

# 4. 목표 불량률 달성을 위한 표준편차
# 규격 내 비율 = 1 - target_defect_rate = 0.99
# P(90 <= X <= 110) = 0.99
# 대칭이므로 P(X <= 110) = 0.995
# z = (110 - 100) / sigma_new = ppf(0.995)
z_critical = stats.norm.ppf(1 - target_defect_rate/2)  # 양측 검정
sigma_required = (upper_spec - mu) / z_critical

print(f"\n4. 불량률 1% 미만을 위한 표준편차: {sigma_required:.2f}g")
print(f"   현재 {sigma}g -> {sigma_required:.2f}g로 {(sigma - sigma_required)/sigma:.1%} 감소 필요")

In [ ]:
# 검증: 새 표준편차로 불량률 확인
new_dist = stats.norm(loc=mu, scale=sigma_required)
new_defect_rate = 1 - (new_dist.cdf(upper_spec) - new_dist.cdf(lower_spec))

print(f"\n검증: 새 표준편차 {sigma_required:.2f}g에서 불량률 = {new_defect_rate:.4f}")
assert new_defect_rate < 0.01, "불량률이 1% 미만이어야 함"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 구간 확률로 규격 내 비율 계산
2. 보수 확률로 불량률 계산
3. 불량률 x 생산량 = 예상 불량 수
4. ppf를 역으로 활용하여 필요 표준편차 계산

**핵심 개념**:
- 양측 규격: 상한과 하한 모두 고려
- 6 시그마: 불량률 0.00034%를 목표로 함

**대안**:
```python
# 이분법으로 표준편차 찾기
from scipy.optimize import brentq
def defect_rate_func(s):
    d = stats.norm(mu, s)
    return 1 - (d.cdf(upper_spec) - d.cdf(lower_spec)) - target_defect_rate
sigma_required = brentq(defect_rate_func, 1, 10)
```

**흔한 실수**:
- 양측 vs 단측 혼동
- ppf 사용 시 확률값 계산 오류

**실무 팁**:
- 제조업 품질 관리의 핵심 분석 방법

---

## Q10. 종합 분석 프로젝트 ⭐⭐⭐⭐⭐

**문제**: 콜센터 데이터를 분석하여 다음 질문에 답하세요.

In [ ]:
# Q10 정답
hourly_calls = [8, 12, 15, 18, 22, 25, 28, 30, 25, 22,
                18, 15, 20, 24, 28, 32, 35, 30, 25, 20,
                15, 12, 10, 8]

print("콜센터 데이터 종합 분석")
print("=" * 60)

# 1. 시간당 평균 통화량과 분산
mean_calls = np.mean(hourly_calls)
var_calls = np.var(hourly_calls, ddof=1)

print(f"\n1. 기술 통계")
print(f"   시간당 평균 통화: {mean_calls:.1f}건")
print(f"   분산: {var_calls:.1f}")
print(f"   표준편차: {np.sqrt(var_calls):.1f}건")

In [ ]:
# 2. 포아송분포 적합성 검증
print(f"\n2. 포아송분포 적합성")
print(f"   평균: {mean_calls:.1f}")
print(f"   분산: {var_calls:.1f}")
print(f"   분산/평균 비율: {var_calls/mean_calls:.2f}")

if 0.8 < var_calls/mean_calls < 1.2:
    print("   -> 포아송분포 적합 (평균 = 분산 근접)")
else:
    print("   -> 포아송분포 부적합 (과분산 또는 과소분산)")
    print("   -> 음이항분포 또는 다른 분포 고려")

In [ ]:
# 3. 20건 이상 통화 확률 (평균 기준 포아송분포 가정)
call_dist = stats.poisson(mean_calls)

prob_20_plus = 1 - call_dist.cdf(19)
print(f"\n3. 20건 이상 통화 확률: {prob_20_plus:.2%}")

In [ ]:
# 4. 95% 신뢰도로 처리 가능한 통화량
capacity_95 = call_dist.ppf(0.95)
print(f"\n4. 95% 신뢰도 처리 용량: {capacity_95:.0f}건/시간")
print(f"   (시간당 {capacity_95:.0f}건까지 처리하면 95% 대응 가능)")

In [ ]:
# 5. Plotly 시각화
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('시간대별 통화량', '포아송분포 vs 실제 분포'))

# 시간대별 통화량 막대그래프
fig.add_trace(
    go.Bar(x=list(range(24)), y=hourly_calls, marker_color='steelblue', name='통화량'),
    row=1, col=1
)

# 포아송분포 vs 실제 분포 비교
x_vals = np.arange(0, 50)
poisson_pmf = stats.poisson.pmf(x_vals, mean_calls)

# 실제 데이터 히스토그램 (정규화)
hist, bins = np.histogram(hourly_calls, bins=range(0, 50), density=True)
bin_centers = (bins[:-1] + bins[1:]) / 2

fig.add_trace(
    go.Bar(x=bin_centers, y=hist, marker_color='lightblue', name='실제 분포', opacity=0.7),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=x_vals, y=poisson_pmf, mode='lines', name='포아송 이론',
               line=dict(color='red', width=2)),
    row=1, col=2
)

fig.update_layout(height=400, width=900, title_text='콜센터 통화량 분석')
fig.update_xaxes(title_text='시간 (0-23시)', row=1, col=1)
fig.update_xaxes(title_text='통화 건수', row=1, col=2)
fig.update_yaxes(title_text='통화 건수', row=1, col=1)
fig.update_yaxes(title_text='확률', row=1, col=2)

fig.show()

In [ ]:
# 비즈니스 인사이트
peak_hour = np.argmax(hourly_calls)
low_hour = np.argmin(hourly_calls)

print("\n" + "=" * 60)
print("비즈니스 인사이트")
print("=" * 60)
print(f"피크 시간대: {peak_hour}시 ({hourly_calls[peak_hour]}건)")
print(f"한가한 시간대: {low_hour}시 ({hourly_calls[low_hour]}건)")
print(f"\n권장 인력 배치:")
print(f"  - 피크 시간: 최소 {int(capacity_95 / 4)}명 (1인당 4건 처리 가정)")
print(f"  - 한가한 시간: 최소 {int(hourly_calls[low_hour] / 4)}명")

In [ ]:
# 테스트
assert 15 < mean_calls < 25, "평균이 예상 범위를 벗어남"
assert capacity_95 > mean_calls, "95% 용량은 평균보다 커야 함"
print("모든 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 기술 통계로 데이터 특성 파악
2. 평균=분산 확인으로 포아송분포 적합성 검증
3. 포아송분포로 확률 계산
4. ppf로 용량 계획
5. 시각화로 인사이트 도출

**핵심 개념**:
- 포아송분포: 기댓값 = 분산 (overdispersion 시 음이항분포)
- 용량 계획: ppf(0.95)로 95% 대응 가능 용량

**대안**:
```python
# 시간대별로 다른 lambda 사용 (비정상 포아송 과정)
# 피크/비피크 시간대 분리 분석
```

**흔한 실수**:
- 데이터의 시간적 패턴 무시
- 포아송분포 가정의 검증 없이 사용

**실무 팁**:
- 콜센터, 서버 용량, 재고 관리에 포아송분포 활용
- 과분산 시 음이항분포 또는 혼합 모델 고려

---

## 📊 학습 정리

### 핵심 정리

| 분포 | 주요 메소드 | 사용 상황 |
|------|------------|----------|
| 정규분포 | `norm.pdf()`, `norm.cdf()`, `norm.ppf()` | 연속 측정값, 품질 관리 |
| 이항분포 | `binom.pmf()`, `binom.cdf()` | 성공/실패 횟수 |
| 포아송분포 | `poisson.pmf()`, `poisson.cdf()` | 희귀 사건 발생 횟수 |

### 확률 계산 공식

| 계산 | 공식 |
|------|------|
| P(X = k) | `pmf(k)` (이산형) |
| P(X <= k) | `cdf(k)` |
| P(X >= k) | `1 - cdf(k-1)` |
| P(a <= X <= b) | `cdf(b) - cdf(a)` |
| 상위 α% 기준점 | `ppf(1-α)` |